In [4]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'
plt.rcParams["figure.dpi"] = 70

base_dir = Path().resolve().parent
data_dir = base_dir / 'data'
sys.path.append(str(base_dir))

In [49]:
data = pd.read_excel(data_dir / 'Project 2_2022.xlsx')
features = ['Income', 'Outcome']  # Условные фичи
target = 'Balance'

## Встроенный метод

Автоматические методы, по сути отбирающие признаки во время обучения (L1, L2 регуляризации)

Добавить при выборе основной модели

## Фильтрационный метод - Correlation-based Feature Selection

In [51]:
from tslib.feature_selection import get_best_cfs_features

print(get_best_cfs_features.__doc__)
get_best_cfs_features(data, features, target)


    Нахождение лучшего сабсета признаков на основе CFS - Correlation-based Feature Selection.
    По сути ищем признаки, сильно коррелирующие с таргетом, и слабо коррелирубщие между собой.
    Для нахождение сабсета используется жадный алгоритм, аналогичный sklearn.feature_selection.SequentialFeatureSelector с direction='forward'
    


(['Outcome'], 0.37111204779131934)